In [ ]:
import tensorflow as tf
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, losses
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Model

In [74]:
(x_train, _), (x_test, _) = fashion_mnist.load_data()
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.

print (x_train.shape)
print (x_test.shape)

(60000, 28, 28)
(10000, 28, 28)


In [ ]:
class Autoencoder(Model):
    def __init__(self, input_dim, latent_dim, encoder_activation, decoder_activation):
        super(Autoencoder, self).__init__()
        self.input_dim = input_dim
        self.latent_dim = latent_dim
        self.encoder_activation = encoder_activation
        self.decoder_activation = decoder_activation

        self.encoder = tf.keras.Sequential([
            # layers.Input(shape=(input_dim,)),
            layers.Dense(latent_dim, activation=self.encoder_activation),
        ])
        self.decoder = tf.keras.Sequential([
            layers.Dense(input_dim, activation=self.decoder_activation),
        # layers.Reshape((28, 28))
        ])

    def encode(self, x):
        return self.encoder(x)

    def decode(self, x):
        return self.decoder(x)

    def call(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded


In [ ]:
# declare variables
latent_dim = 64
input_dim = 784

autoencoder = Autoencoder(input_dim, latent_dim, 'relu', 'sigmoid')
autoencoder.compile(optimizer='adam', loss=losses.MeanSquaredError())
autoencoder.fit(x_train, x_train, epochs=10, shuffle=True, validation_data=(x_test, x_test))

In [62]:
class SDAE(Model):
    def __init__(self, ae_layers):
        super(SDAE, self).__init__()
        self.ae_layers = ae_layers
        self.models = []

    def make(self):
        for i, layer in enumerate(self.ae_layers[:-1]):
            print('building layer input {} output {}'.format(layer, self.ae_layers[i+1]))
            m = Autoencoder(layer, self.ae_layers[i+1], 'relu', 'sigmoid')
            # m.compile(optimizer='adam', loss=losses.MeanSquaredError())
            self.models.append(Autoencoder(layer, self.ae_layers[i+1], 'relu', 'sigmoid'))

    def call(self, train, test, epochs):
        train_set = train
        test_set = test

        for m in self.models:
            m.compile(optimizer='adam', loss=losses.MeanSquaredError())
            m.fit(train_set, train_set, epochs=epochs, shuffle=True, validation_data=(test_set, test_set))
            train_set = m.encode(train_set)
            test_set = m.encode(test_set)            

    def get_layers(self):
        model_layers = []
        for m in self.models:
            w = m.get_weights()
            layer_dict = {
                'w1': w[0],
                'b1': w[1],
                'w2': w[2],
                'b2': w[3]
            }
            model_layers.append(layer_dict)
        return model_layers

In [ ]:
print(sdae.get_layers())



In [ ]:
ae_layers = [784, 64, 16]

print(x_train.shape)
print(x_test.shape)

x_train = x_train.reshape(60000, 784)
x_test= x_test.reshape(10000, 784)

sdae = SDAE(ae_layers)
sdae.make()
# sdae.compile(optimizer='adam', loss=losses.MeanSquaredError())
sdae.call(x_train, x_test, epochs=2)

In [ ]:
b = sdae.models[0].get_weights()

for x in b:
    print(x.shape)


In [73]:
sdae = SDAE(ae_layers)
sdae.make()
sdae.call(x_train, x_test, epochs=10)
a = sdae.get_layers()

building layer input 784 output 64
building layer input 64 output 16
Epoch 1/10
1875/1875 [==============================] - 3s 1ms/step - loss: 0.0392 - val_loss: 0.0131
Epoch 2/10
1875/1875 [==============================] - 3s 1ms/step - loss: 0.0122 - val_loss: 0.0106
Epoch 3/10
1875/1875 [==============================] - 3s 1ms/step - loss: 0.0101 - val_loss: 0.0096
Epoch 4/10
1875/1875 [==============================] - 3s 1ms/step - loss: 0.0095 - val_loss: 0.0094
Epoch 5/10
1875/1875 [==============================] - 3s 1ms/step - loss: 0.0092 - val_loss: 0.0092
Epoch 6/10
1875/1875 [==============================] - 3s 1ms/step - loss: 0.0089 - val_loss: 0.0090
Epoch 7/10
1875/1875 [==============================] - 3s 1ms/step - loss: 0.0089 - val_loss: 0.0089
Epoch 8/10
1875/1875 [==============================] - 3s 1ms/step - loss: 0.0088 - val_loss: 0.0088
Epoch 9/10
1875/1875 [==============================] - 3s 1ms/step - loss: 0.0087 - val_loss: 0.0090
Epoch 10/10
1

In [72]:
a[0]['w1'].shape

(784, 64)

In [83]:
x_train = x_train.reshape(60000, 784)

print(x_train.shape)
split = int(x_train.shape[0] * .8)
print(split)
print(x_train[:split].shape)
print(x_train[split:].shape)




(60000, 784)
48000
(48000, 784)
(12000, 784)
